In [ ]:
#### 
###   Parse data from CAISO website using my credentials
###   Login: yurymaximov@berkeley.edu   Password: vodka8Pivo_
###
###
###   Input: 
###     date_start - start time for RTM/DAM market price analysis (format: '2022-07-01')
###     days   - days to analyse (format: 31, i.e. from July 1st to July 31st of 2022)
###
###   Output:
###     5 mins schedule of locational marginal prices for a given time interval
###
###   Abbreviations:
###     DAM - day ahead market
###     RTM - real time market (5 minutes schedule)

In [ ]:
## User input

# Start date for the analysis
date_start = '2022-02-01'

# Days (interval: date_start to date_start + days)
days = 7

# Directory for datasets
root_path = 'gdrive/My Drive/TechnoEconomics'

In [ ]:
## Import required libraries and APIs

import glob
import io
import os
import time
import zipfile
from dateutil import rrule
from datetime import datetime, timedelta, date
import requests
import pandas as pd

## Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
## Convert the date com internally compatible format
##
## TODO: date to iso
def get_date(start_date, numofdays: int):
    now = date.fromisoformat(start_date)
    end_date = now + timedelta(days=numofdays)

    listofdays = rrule.rrule(rrule.DAILY, dtstart=now, until=end_date)
    listofdays = list(map(str, listofdays))
    listofdays = list(
        map(lambda x: x.replace('-', '').replace(' ', 'T').rstrip(listofdays[0][14:]) + '00:00-0000', listofdays))
    return listofdays

In [ ]:
##  Concatenate all data and write as a single file
def concate_all(path):
    all_files = glob.glob(os.path.join(path, "*.csv"))

    li = []
    print(all_files)
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0, low_memory=False)
        li.append(df)

    frame = pd.concat(li, axis=0, ignore_index=True)
    
    ## need to check
    frame.to_csv(root_path+'multi.csv', index=False)

In [ ]:
## Function benchmarker wrapper
def benchmark(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        print('success', *args)

    return wrapper

In [ ]:
## CAISO Local marginal prices (Real time market = RTM)
def get_prices(nodename, startdate, enddate):
    rsp = requests.get(
        f'http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_INTVL_LMP&startdatetime={startdate}&enddatetime={enddate}&version=1&market_run_id=RTM&node={nodename}&resultformat=6',
        timeout=335)

    z = zipfile.ZipFile(io.BytesIO(rsp.content))
    csv = z.open(z.namelist()[0])
    df = pd.read_csv(csv)
    return df

In [ ]:
## CAISO Local marginal prices (Day ahead market = DAM)
def get_dam(nodename, startdate, enddate):
    # http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_LMP&market_run_id=DAM&startdatetime=20210101T08%3A00-0000&enddatetime=20210102T08%3A00-0000&version=1&node=CAPTJACK_5_N003&resultformat=6

    rsp = requests.get(
        f'http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_LMP&startdatetime={startdate}&enddatetime={enddate}&version=1&market_run_id=DAM&node={nodename}&resultformat=6')
    z = zipfile.ZipFile(io.BytesIO(rsp.content))
    csv = z.open(z.namelist()[0])
    df = pd.read_csv(csv)
    return df


In [ ]:
## Additional node information (any existing, RTM market)
def get_node_info(name, dates, path):
    list_of_csv = []
    for day in range(len(dates) - 1):
        time.sleep(6)
        df = get_prices(name, dates[day], dates[day + 1])
        print(df.head())
        list_of_csv.append(df)
    print(len(list_of_csv))

    df_f = pd.concat(list_of_csv, ignore_index=True)
    print('Success')
    df_f.to_csv(f'{path}/{name}.csv', index=False)

In [ ]:
## Additional node information (any existing, DAM market)
def get_node_DAM(name, dates, path):
    list_of_csv = []
    for day in range(len(dates) - 1):
        time.sleep(6)
        df = get_dam(name, dates[day], dates[day + 1])
        print(df.head())
        list_of_csv.append(df)
    print(len(list_of_csv))

    df_f = pd.concat(list_of_csv, ignore_index=True)
    print('Success')
    df_f.to_csv(f'{path}/{name}_DAM.csv', index=False)

In [ ]:
# get CO2 emissions information
def getco2(listofdates):
    for dt in listofdates:
        datetoreq = dt[:8]
        print(datetoreq)
        rsp = requests.get(f'https://www.caiso.com/outlook/SP/History/{datetoreq}/co2.csv?=1659725917675')
        df = pd.read_csv(io.StringIO(rsp.content.decode('utf-8')))
        df = df.rename(columns={'Time': 'Date'})

        df['Date'] = dt[:9] + df['Date'].astype(str)
        # df['Date']='20210701T'+df['Date'].str.replace(',','-')
        print('Success')
        df.to_csv(f'outputs/{dt}.csv', index=False)
        time.sleep(5)

In [ ]:
# Get prices ++ from CAISO HOLLISTR node RTM market
# 
get_node_info('HOLLISTR_1_N101', get_date(date_start, days), root_path)

       INTERVALSTARTTIME_GMT        INTERVALENDTIME_GMT      OPR_DT  OPR_HR  \
0  2022-02-01T00:00:00-00:00  2022-02-01T00:05:00-00:00  2022-01-31      17   
1  2022-02-01T00:05:00-00:00  2022-02-01T00:10:00-00:00  2022-01-31      17   
2  2022-02-01T00:10:00-00:00  2022-02-01T00:15:00-00:00  2022-01-31      17   
3  2022-02-01T00:15:00-00:00  2022-02-01T00:20:00-00:00  2022-01-31      17   
4  2022-02-01T00:20:00-00:00  2022-02-01T00:25:00-00:00  2022-01-31      17   

       NODE_ID_XML          NODE_ID             NODE MARKET_RUN_ID LMP_TYPE  \
0  HOLLISTR_1_N101  HOLLISTR_1_N101  HOLLISTR_1_N101           RTM      MCC   
1  HOLLISTR_1_N101  HOLLISTR_1_N101  HOLLISTR_1_N101           RTM      MCC   
2  HOLLISTR_1_N101  HOLLISTR_1_N101  HOLLISTR_1_N101           RTM      MCC   
3  HOLLISTR_1_N101  HOLLISTR_1_N101  HOLLISTR_1_N101           RTM      MCC   
4  HOLLISTR_1_N101  HOLLISTR_1_N101  HOLLISTR_1_N101           RTM      MCC   

  XML_DATA_ITEM    PNODE_RESMRID GRP_TYPE  POS    

In [ ]:
# Get prices ++ from CAISO HOLLISTR node DAM market
#

#get_node_DAM('HOLLISTR_1_N101', get_date(date_start, days), root_path)

In [ ]:
    getco2(get_date('2022-02-01', 7))
    #print(get_date('2021-08-05',1))
    # concate_all(path)

20220201
Success


FileNotFoundError: ignored

In [ ]:
## Hardcoded request -- prices from 2021-08-01 to 2022-07-31
from tqdm import tqdm

def my_req_int():
    df = pd.read_csv('LMPLocations.csv')
    names_lst = df['name'].tolist()

    for name in names_lst:
        time.sleep(6)
        df1 = get_prices(name, '20210801T00:00-0000', '20210831T00:00-0000')
        time.sleep(6)
        
        df2 = get_prices(name, '20210901T00:00-0000', '20210930T00:00-0000')
        time.sleep(6)
        
        df3 = get_prices(name, '20211001T00:00-0000', '20211031T00:00-0000')
        time.sleep(6)
        
        df4 = get_prices(name, '20211101T00:00-0000', '20211130T00:00-0000')
        time.sleep(6)

        df5 = get_prices(name, '20211201T00:00-0000', '20211231T00:00-0000')
        time.sleep(6)

        df6 = get_prices(name, '20220101T00:00-0000', '20220131T00:00-0000')
        time.sleep(6)

        df7 = get_prices(name, '20220201T00:00-0000', '20220228T00:00-0000')
        time.sleep(6)

        df8 = get_prices(name, '20220301T00:00-0000', '20220331T00:00-0000')
        time.sleep(6)

        df9 = get_prices(name, '20220401T00:00-0000', '20220430T00:00-0000')
        time.sleep(6)

        df10 = get_prices(name, '20220501T00:00-0000', '20220531T00:00-0000')
        time.sleep(6)
        
        df11 = get_prices(name, '20220601T00:00-0000', '20220630T00:00-0000')
        time.sleep(6)
        
        df12 = get_prices(name, '20220701T00:00-0000', '20220731T00:00-0000')
        time.sleep(6)

        df_f = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12], ignore_index=True)
        print('Success')
        df_f.to_csv(f'csv/{name}.csv', index=False)